In [2]:
import json, os

In [40]:
data = {}

def compound_dicts(dicts: list[dict]):
    return {k: v for d in dicts for k, v in d.items()}


def diff_dict(d1, d2):
    result = {}
    for key in d2:
        if key not in d1 or d1[key] != d2[key]:
            result[key] = d2[key]

    return result


def walk_data(input_data, date):
    for item in input_data:
        # walk categories recursively
        if 'categories' in item:
            walk_data(item["categories"], date)
            continue
        if "products" in item:
            walk_data(item["products"], date)
            continue
        if "content" in item: # old api uses "content" instead of "categories" and "products"
            walk_data(item["content"], date)
            continue
        
        try:       
            # product

            id = str(item['id'])

            values = {
                "iva": item["iva"],
                "unit_size": item["unit_size"],
                "bulk_price": item["bulk_price"],
                "unit_price": item["unit_price"],
                "size_format": item["size_format"]
                }
            

            if data.get(id):
                last_values = compound_dicts(data[id]["data"].values())
                diff = diff_dict(last_values, values) # values that have changed

                if diff:
                    data[id]["data"][date] = diff

            else:
                data[id] = {"name": item["name"], "data": {date: values}}


        except:
            print(item)
            print(file)
            raise Exception("Error")



for file in os.listdir('../data'):
    with open('../data/' + file, 'r', encoding='utf-8') as f:
        file_data = json.load(f)

        walk_data(file_data, file.split('.')[0])


with open('../data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent='\t', ensure_ascii=False)